### Capstone Project Edward Garzon

### Import pandas costal code Canada and transform data

In [1]:
import numpy as np
import pandas as pd

In [55]:
# The code was removed by Watson Studio for sharing.

In [29]:
df=pd.DataFrame(df_data_3.Name.str.split(';',2).tolist(), 
                         columns = ['Postal Code','Borough','Neighbourhood'])

In [52]:
df= df[df['Borough'] != "Not assigned"]

In [33]:
df1=df.groupby("Postal Code").agg(lambda x:','.join(set(x)))

In [53]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")

In [54]:
df1['Latitude']=geo_data['Latitude'].values
df1['Longitude']=geo_data['Longitude'].values

In [47]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [48]:
CLIENT_ID = 'J1Z40IDNZIEGVMUQSMLERIA20YZPQAIWYSGIZHY1WT5NQY1H' # your Foursquare ID
CLIENT_SECRET = 'EVGWNXALHD3SS4XCLUZAQWUKAYK4WA1WSXLHGJWP23FTYSXM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J1Z40IDNZIEGVMUQSMLERIA20YZPQAIWYSGIZHY1WT5NQY1H
CLIENT_SECRET:EVGWNXALHD3SS4XCLUZAQWUKAYK4WA1WSXLHGJWP23FTYSXM


### I decide looking for a museums Near of 111 Queens Park, Toronto, CA 

In [51]:
address = '111 Queens Park, Toronto, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.659659 -79.3903399


In [56]:
search_query = 'Museum'
radius = 500
print(search_query + ' .... OK!')

Museum .... OK!


In [57]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=J1Z40IDNZIEGVMUQSMLERIA20YZPQAIWYSGIZHY1WT5NQY1H&client_secret=EVGWNXALHD3SS4XCLUZAQWUKAYK4WA1WSXLHGJWP23FTYSXM&ll=43.659659,-79.3903399&v=20180604&query=Museum&radius=500&limit=30'

In [59]:
results = requests.get(url).json()

In [60]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",False,4ad4c05ef964a520e2f620e3,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,662,[55 Centre Avenue (University Ave. and Dundas ...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,M5G 2H5,ON,Textile Museum of Canada,v-1595427053,56305293
1,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",False,4ad4c05ef964a520dcf620e3,40 College St,CA,Toronto,Canada,in Toronto Police Headquarters,484,[40 College St (in Toronto Police Headquarters...,"[{'label': 'display', 'lat': 43.66118880223964...",43.661189,-79.384703,M5G 2J3,ON,Toronto Police Museum,v-1595427053,NaN


In [61]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Textile Museum of Canada,Art Museum,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,662,[55 Centre Avenue (University Ave. and Dundas ...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,M5G 2H5,ON,4ad4c05ef964a520e2f620e3
1,Toronto Police Museum,History Museum,40 College St,CA,Toronto,Canada,in Toronto Police Headquarters,484,[40 College St (in Toronto Police Headquarters...,"[{'label': 'display', 'lat': 43.66118880223964...",43.661189,-79.384703,M5G 2J3,ON,4ad4c05ef964a520dcf620e3


In [62]:
dataframe_filtered.name

0    Textile Museum of Canada
1       Toronto Police Museum
Name: name, dtype: object

In [63]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map